In [ ]:

# https://www.tensorflow.org/tutorials/images/classification

import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import PIL
import tensorflow as tf

from tensorflow import keras 
from keras import layers  #pas de tensorflow.kerras
from keras.models import Sequential #idem
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)


image_count = len(list(data_dir.glob('*/*.jpg')))

print('data_dir', data_dir,'image_count', image_count)
varroas = list(data_dir.glob('varroas/*'))
PIL.Image.open(str(varroas[0]))

batch_size = 200
img_height = 22
img_width = 22
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

for image_batch, labels_batch in train_ds: #image_batch ?
  print(image_batch.shape)
  print(labels_batch.shape)
  break

# AUTOTUNE = tf.data.AUTOTUNE

# train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

num_classes = len(class_names)

# Augmentation des données
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)


# Créons un nouveau réseau de neurones avec tf.keras.layers.Dropout 
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

# Compiler et entraîner le modèle
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()
epochs = 200 # 200
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

# Visualisez les résultats de l'entraînement
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# sauvegarde du model 
# !mkdir -p saved_model
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model_200_1.h5')
# Prédire sur de nouvelles données


Ce code est une implémentation d'un modèle de classification d'images à l'aide de TensorFlow et Keras. Il est écrit en Python.

Le code commence par importer les bibliothèques nécessaires, notamment Matplotlib, NumPy, PIL, TensorFlow et Keras. Il définit également la taille de l'image d'entrée, la taille du lot et les répertoires de formation et de validation.

Il utilise image_dataset_from_directory pour charger les données d'entraînement et de validation à partir des répertoires respectifs. Il normalise également les images en utilisant Rescaling et applique l'augmentation de données avec RandomFlip, RandomRotation, et RandomZoom.

Le modèle est construit à l'aide de Sequential et comprend plusieurs couches de convolution et de regroupement, une couche de baisse aléatoire et deux couches denses. Il utilise l'optimiseur Adam et la fonction de perte SparseCategoricalCrossentropy pour la classification multiclasse.

Le modèle est ensuite entraîné avec les données d'entraînement et de validation, et l'historique d'entraînement est stocké pour l'analyse. Enfin, les résultats d'entraînement sont tracés à l'aide de Matplotlib et le modèle est enregistré.

Notez que certaines parties du code sont commentées. Par exemple, le code pour télécharger les images de fleurs est en commentaire, car il est supposé que les images ont déjà été téléchargées et sont disponibles dans le répertoire spécifié.

RANDOM FLIP

Le 'random flip' est appliqué de manière aléatoire sur les images en entrée. Cela signifie que pour chaque image, la fonction de transformation 'RandomFlip' décide si elle doit être retournée horizontalement (de gauche à droite) ou non. Cette transformation est une technique d'augmentation de données qui permet d'augmenter la diversité des images disponibles pour l'apprentissage et ainsi d'améliorer la capacité du modèle à généraliser sur de nouvelles données.

RANDOM ROTATION

La fonction de transformation 'RandomRotation' est appliquée de manière aléatoire sur les images en entrée. Elle effectue une rotation aléatoire d'un angle compris entre -10 et 10 degrés, ce qui permet d'augmenter la diversité des images et donc d'améliorer la capacité du modèle à généraliser sur de nouvelles données. Cette transformation est également une technique d'augmentation de données couramment utilisée en vision par ordinateur.

Random ZOOM

La fonction de transformation 'RandomZoom' est également appliquée de manière aléatoire sur les images en entrée. Elle effectue un zoom aléatoire sur les images en entrée, en agrandissant ou en réduisant leur taille de manière aléatoire d'un facteur compris entre 0 et 10%. Cette transformation est une autre technique d'augmentation de données couramment utilisée en vision par ordinateur pour améliorer la robustesse du modèle à des variations d'échelle dans les images.

SEQUENTIAL

En utilisant le module "Sequential" de la bibliothèque Keras, on peut créer des modèles de réseau de neurones en séquentiel, c'est-à-dire que les couches du modèle sont ajoutées les unes après les autres, dans l'ordre dans lequel elles doivent être exécutées.

Dans le code donné, le modèle est créé à l'aide de "Sequential", et les différentes couches du réseau de neurones sont ajoutées les unes après les autres en utilisant la méthode "add". Ce modèle séquentiel est ensuite compilé et entraîné.

image_batch label_batch

image_batch et label_batch sont des variables contenant un batch d'images et les étiquettes correspondantes (ou labels) respectivement. Ces variables sont créées à partir de la variable train_ds qui est créée en utilisant la fonction tf.keras.utils.image_dataset_from_directory().

Dans ce code, image_batch et label_batch sont utilisées pour vérifier la taille des lots (batch) d'images et de labels. La commande print(image_batch.shape) affiche la forme du batch d'images, qui devrait être de la forme (batch_size, img_height, img_width, num_channels), où batch_size est la taille du batch, img_height et img_width sont les dimensions des images, et num_channels est le nombre de canaux de couleur (3 pour une image en couleur RGB). De même, la commande print(labels_batch.shape) affiche la forme du batch de labels, qui devrait être de la forme (batch_size,).